In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("./DATA/training.csv", encoding='ISO8859-2')
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,\nMore\nI can't make any real suggestions on i...,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
127652,ffe8b9316245be30,The numbers in parentheses are the additional ...,0,0,0,0,0,0
127653,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
127654,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
127655,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=10)

X_tfidf = tfidf.fit_transform(data['comment_text'].values)
X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names())
X_df

,00,000,0000,00000,000000,0000000,00000000,0000000027,00000001,00000003,...,ߞދ,ߞދӈ,ߞߞ,ߞߟ,ߟӈ,ߟދ,ߧz,੧z,ఽg,덡h
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
X_topics = pca.fit_transform(X_df)
X_topics

In [1]:
y_columns = data.columns[2:]
models = []

NameError: name 'data' is not defined

In [42]:
from sklearn.model_selection import train_test_split
for y_column in y_columns:
    y = data[y_column]
    X_train, X_valid, y_train, y_valid = train_test_split(X_topics, y, test_size=0.25, stratify=y)
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import f1_score
    rfc =RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_valid_pred = rfc.predict(X_valid)
    models.append(rfc)
    print("%s f1_score: %.2f" % (y_column, f1_score(y_valid, y_valid_pred) * 100))

toxic f1_score: 34.83
severe_toxic f1_score: 10.50
obscene f1_score: 29.79
threat f1_score: 0.00
insult f1_score: 22.11
identity_hate f1_score: 3.30


In [39]:
test = pd.read_csv("./DATA/test.csv", encoding='ISO8859-2')
test_tfidf = tfidf.transform(test['comment_text'].values)
test_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names())

In [66]:
test_topics = pca.transform(test_df)
result = test['id']
for (idx, column) in enumerate(y_columns):
    y_test = models[idx].predict(test_topics)
    result = pd.concat([result, pd.DataFrame(y_test, columns=[column], index=test.index)], axis=1)

In [68]:
result.to_csv("_results.csv", index=None)